### Import required libraries

In [25]:
# Install required libraries
%pip install -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (

#### Access LangSmith

What is LangSmith?
- A platform for building production grade LLM applications
- Compare results across models, architectures and prompts to identify what works best
- Quickly refine prompts to achieve more accurate and reliable results
- LangSmith integrate seamlessly with LangChain's open source frameworks, *langchain* and *langgraph* with no extra instrumentation needed.

https://docs.smith.langchain.com

In [20]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Enables LangChain Tracing V2, a mechanism to enable and monitor the execution of 
## LangChain applications for debugging and optimization
# Configures the LangChain API End Point to use LangChain for tracing
# API key provides authentication to access LangChain service 
os.environ['LANGCHAIN_TRACING_V2'] = 'true' 
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

#### Load Documents

Why Use Beautiful Soap Python package?
- To scrape data from web pages.
- To clean and restructure HTML or XML documents.
- To find elements using tags, classes, or attributes.

In [15]:
import bs4 # Import Beautiful Soap, a Python Web Scrapping package

# Load Documents
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load() 

#### Split the documents 

In [18]:
# Split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

#### Get the embeddings of the splits, store in the VectorStore & convert the Vector Store into a retriever object 

In [24]:
# Import Chroma Vector Store
from langchain_community.vectorstores import Chroma # Import Chroma
from langchain_openai import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.